In [1]:
import pandas as pd
import os
import numpy as np

In [42]:
path="../data/usermodel/"
files=os.listdir(path)
sorted_files=sorted(files, reverse=True)
print(sorted_files)

['20231101.txt', '20231001.txt', '20230901.txt', '20230801.txt']


In [43]:
list_months=[]
df_list=[]
for file in sorted_files[:4]:
    print(file)
    month="t"+file.split(".")[0][4:6:1]
    list_months.append(month)
    df=pd.read_csv(path+file, usecols=['isdn', 'revenue', 'arpu',
       'td_thoai', 'value_thoai', 'count_thoai', 'td_data', 'value_data',
       'count_data', 'td_sms', 'value_sms', 'count_sms', 'td_gtgt', 'td_rmqt',
       'td_other', 'partition'], sep=',')
    df.columns=['{}_{}'.format(col, month) for col in df.columns]
    df_list.append(df)
#print(list_months)

20231101.txt
20231001.txt
20230901.txt
20230801.txt


In [44]:
df_final=pd.concat([x.set_index(x.columns[0]) for x in df_list[:4]], axis=1, join='inner')

In [45]:
drop_fields=['partition']

In [46]:
df_final.drop(columns=['{}_{}'.format(title,month) for title in drop_fields for month in list_months], inplace=True)

In [47]:
df_final.columns

Index(['revenue_t11', 'arpu_t11', 'td_thoai_t11', 'value_thoai_t11',
       'count_thoai_t11', 'td_data_t11', 'value_data_t11', 'count_data_t11',
       'td_sms_t11', 'value_sms_t11', 'count_sms_t11', 'td_gtgt_t11',
       'td_rmqt_t11', 'td_other_t11', 'partition_t11', 'revenue_t10',
       'arpu_t10', 'td_thoai_t10', 'value_thoai_t10', 'count_thoai_t10',
       'td_data_t10', 'value_data_t10', 'count_data_t10', 'td_sms_t10',
       'value_sms_t10', 'count_sms_t10', 'td_gtgt_t10', 'td_rmqt_t10',
       'td_other_t10', 'revenue_t09', 'arpu_t09', 'td_thoai_t09',
       'value_thoai_t09', 'count_thoai_t09', 'td_data_t09', 'value_data_t09',
       'count_data_t09', 'td_sms_t09', 'value_sms_t09', 'count_sms_t09',
       'td_gtgt_t09', 'td_rmqt_t09', 'td_other_t09', 'revenue_t08', 'arpu_t08',
       'td_thoai_t08', 'value_thoai_t08', 'count_thoai_t08', 'td_data_t08',
       'value_data_t08', 'count_data_t08', 'td_sms_t08', 'value_sms_t08',
       'count_sms_t08', 'td_gtgt_t08', 'td_rmqt_t08

In [48]:
# lấy lại column index=isdn
df_final.reset_index(inplace=True)

In [49]:
df_final.rename(columns={"index":"isdn"}, inplace=True)

In [50]:
df_final.head()

,isdn,revenue_t11,arpu_t11,td_thoai_t11,value_thoai_t11,count_thoai_t11,td_data_t11,value_data_t11,count_data_t11,td_sms_t11,...,count_thoai_t08,td_data_t08,value_data_t08,count_data_t08,td_sms_t08,value_sms_t08,count_sms_t08,td_gtgt_t08,td_rmqt_t08,td_other_t08
0,906994834,0.00,0,0.0,0,0,0.00,0,0,0.00,...,0,0.00,0,0,0.0,0,0,200.0,0.0,0.0
1,906995737,23940.00,28326,11940.0,449,24,0.00,0,0,0.00,...,31,0.00,0,0,0.0,0,0,12158.0,0.0,0.0
2,906998123,161099.98,172227,21900.0,6187,87,128099.99,35342212891,0,2099.99,...,187,148499.98,57005111,0,640.0,0,2,9200.0,0.0,0.0
3,906998570,90000.00,90000,810.0,505,33,89190.00,17540767595,0,0.00,...,33,179190.00,24887045,0,0.0,0,0,0.0,0.0,0.0
4,907002359,0.00,0,0.0,0,0,0.00,0,0,0.00,...,0,0.00,0,0,0.0,0,0,0.0,0.0,0.0


In [51]:
df_final.to_csv("../output/data_raw_{}.csv".format(list_months[0]), index=False)

In [14]:
df_final=pd.read_csv("../output/data_raw_{}.csv".format("t11") , sep=",")

In [15]:
# compute arpu_data, arpu_data_thoai of cur_month
def compute_arpu(df_month,list_months, cur_month):
    title1="arpu_data_"+cur_month
    title2="arpu_thoai_"+cur_month
    title3="arpu_data_thoai_"+cur_month
    df_month[title1]=0
    df_month[title2]=0
    for month in list_months:
        df_month[title1]+=df_month["td_data_"+month]
        df_month[title2]+=df_month["td_thoai_"+month]
    df_month[title1]=(df_month[title1])/3
    df_month[title2]=(df_month[title2])/3
    df_month[title3]=df_month[title1]+df_month[title2]

In [16]:
# compute td_data_thoai of month in list_months
def compute_td(df_month, list_months):
    for month in list_months:
        col1="td_data_thoai_"+month
        col_data="td_data_"+month
        col_thoai="td_thoai_"+month
        df_month[col1]=df_month[col_data]+df_month[col_thoai]

In [17]:
list_months=["t11","t10","t09","t08"]

In [18]:
compute_arpu(df_final, list_months[:3], list_months[0])
compute_arpu(df_final, list_months[1:], list_months[1])
compute_td(df_final, list_months[1:])

In [7]:
df_final.columns

Index(['isdn', 'revenue_t11', 'arpu_t11', 'td_thoai_t11', 'value_thoai_t11',
       'count_thoai_t11', 'td_data_t11', 'value_data_t11', 'count_data_t11',
       'td_sms_t11', 'value_sms_t11', 'count_sms_t11', 'td_gtgt_t11',
       'td_rmqt_t11', 'td_other_t11', 'partition_t11', 'revenue_t10',
       'arpu_t10', 'td_thoai_t10', 'value_thoai_t10', 'count_thoai_t10',
       'td_data_t10', 'value_data_t10', 'count_data_t10', 'td_sms_t10',
       'value_sms_t10', 'count_sms_t10', 'td_gtgt_t10', 'td_rmqt_t10',
       'td_other_t10', 'revenue_t09', 'arpu_t09', 'td_thoai_t09',
       'value_thoai_t09', 'count_thoai_t09', 'td_data_t09', 'value_data_t09',
       'count_data_t09', 'td_sms_t09', 'value_sms_t09', 'count_sms_t09',
       'td_gtgt_t09', 'td_rmqt_t09', 'td_other_t09', 'revenue_t08', 'arpu_t08',
       'td_thoai_t08', 'value_thoai_t08', 'count_thoai_t08', 'td_data_t08',
       'value_data_t08', 'count_data_t08', 'td_sms_t08', 'value_sms_t08',
       'count_sms_t08', 'td_gtgt_t08', 'td_

In [19]:
# general field name
get_fields=['arpu','revenue','arpu_data_thoai','arpu_data','td_data','td_data_thoai']

In [20]:
# to get list of needed fieldnames
def get_list_fields(get_field, list_fields, list_months):
    for field in get_field:
        f=field+'_{}'
        for month in list_months:
            list_fields.append(f.format(month))

In [21]:
list_fields=['isdn']
get_list_fields(get_fields[:2],list_fields,list_months[:3])
get_list_fields(get_fields[2:4], list_fields,list_months[:2])
get_list_fields(get_fields[4:],list_fields,list_months[1:])
print((list_fields))

['isdn', 'arpu_t11', 'arpu_t10', 'arpu_t09', 'revenue_t11', 'revenue_t10', 'revenue_t09', 'arpu_data_thoai_t11', 'arpu_data_thoai_t10', 'arpu_data_t11', 'arpu_data_t10', 'td_data_t10', 'td_data_t09', 'td_data_t08', 'td_data_thoai_t10', 'td_data_thoai_t09', 'td_data_thoai_t08']


In [26]:
table2=df_final[[col for col in list_fields]]

In [29]:
table2.rename(columns={"revenue_t11":"td_vien_thong_t11","revenue_t10":"td_vien_thong_t10","revenue_t09":"td_vien_thong_t09"}, inplace=True)

/tmp/ipykernel_1979976/1248709724.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table2.rename(columns={"revenue_t11":"td_vien_thong_t11","revenue_t10":"td_vien_thong_t10","revenue_t09":"td_vien_thong_t09"}, inplace=True)


In [27]:
#table2.rename(columns={("revenue_{}".format(month1) for month1 in list_months[:3]):"td_vien_thong_{}".format(month2) for month2 in list_months[:3]},inplace=True)

/tmp/ipykernel_1979976/2173920030.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table2.rename(columns={("revenue_{}".format(month1) for month1 in list_months[:3]):"td_vien_thong_{}".format(month2) for month2 in list_months[:3]},inplace=True)


In [30]:
table2.head(50)

,isdn,arpu_t11,arpu_t10,arpu_t09,td_vien_thong_t11,td_vien_thong_t10,td_vien_thong_t09,arpu_data_thoai_t11,arpu_data_thoai_t10,arpu_data_t11,arpu_data_t10,td_data_t10,td_data_t09,td_data_t08,td_data_thoai_t10,td_data_thoai_t09,td_data_thoai_t08
0,906994834,0,66,133,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00
1,906995737,28326,28244,28950,23940.00,26482.00,34558.00,16160.000000,16025.000000,0.000000,0.000000,0.00,0.00,0.00,14482.00,22058.00,11535.00
2,906998123,172227,195503,202930,161099.98,163219.98,192363.00,151670.996667,175366.990000,128099.990000,134899.986667,128099.99,128099.99,148499.98,149999.99,155013.01,221087.97
3,906998570,90000,120000,120000,90000.00,90000.00,90000.00,90000.000000,120000.000000,89190.000000,119190.000000,89190.00,89190.00,179190.00,90000.00,90000.00,180000.00
4,907002359,0,0,0,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00
5,907007995,4963,4533,1000,1290.00,10600.00,3000.00,3763.333333,3333.333333,3333.333333,3333.333333,10000.00,0.00,0.00,10000.00,0.00,0.00
6,907022261,57678,96326,88878,13825.00,130793.00,28416.00,53292.000000,55274.000000,45316.666667,43875.000000,109525.00,20825.00,1275.00,126393.00,22458.00,16971.00
7,907021252,0,0,0,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00
8,907022360,0,108999,109000,0.00,108999.99,108999.99,72012.660000,108336.180000,52923.126667,88501.813333,79384.69,79384.69,106736.06,108018.99,108018.99,108970.56
9,907026369,0,0,0,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00


In [90]:
table2[table2['isdn']==908558942]

,partition,isdn,td_vien_thong_t11,td_vien_thong_t10,td_vien_thong_t09,arpu_t09,arpu_t11,arpu_t10,arpu_data_thoai_t11,arpu_data_thoai_t10,arpu_data_t11,arpu_data_t10,td_data_t10,td_data_t09,td_data_t08,td_data_thoai_t10,td_data_thoai_t09,td_data_thoai_t08
485408,20231101,908558942,173184.0,120000.0,120000.0,120000,137728,120000,177394.666667,177394.666667,165765.0,165765.0,116640.0,116640.0,116640.0,120000.0,120000.0,120000.0


In [25]:
table2.to_csv("../output/table2_{}.csv".format(list_months[0]), index=False)

In [92]:
del df_final
del table2